# 도로명주소 위경도 좌표변환
- `geopy` 패키지 이용한 지오코딩

In [ ]:
# For Google Colab
from google.colab import drive
drive.mount('/content/Mydrive')
%cd "/content/Mydrive/MyDrive/IOT_hackathon/Github/IOT_hackathon"

Mounted at /content/Mydrive
/content/Mydrive/MyDrive/IOT_hackathon/Github/IOT_hackathon


In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
import numpy as np
from tqdm import tqdm

geo_local = Nominatim(user_agent='South Korea', timeout=None)
df = pd.read_csv('data/seoul_schools.csv', encoding='cp949')
df = df[['학교종류명','도로명주소']]

In [ ]:
df.학교종류명.unique()

array(['각종학교(중)', '초등학교', '중학교', '외국인학교', '평생학교(초)-3년6학기', '각종학교(초)',
       '방송통신중학교', '고등학교', '특수학교', '각종학교(고)', '평생학교(고)-2년6학기',
       '평생학교(고)-3년6학기', '평생학교(중)-2년6학기', '평생학교(중)-3년6학기', '고등기술학교',
       '방송통신고등학교', '공동실습소'], dtype=object)

In [ ]:
# 학교 종류가 많으므로 전처리
df.loc[df.학교종류명 == '각종학교(초)','학교종류명'] = '초등학교'
df.loc[df.학교종류명 == '각종학교(중)','학교종류명'] = '중학교'
df.loc[df.학교종류명 == '각종학교(고)','학교종류명'] = '고등학교'
df.loc[df.학교종류명 == '평생학교(초)-3년6학기','학교종류명'] = '초등학교'
df.loc[df.학교종류명 == '평생학교(중)-2년6학기','학교종류명'] = '중학교'
df.loc[df.학교종류명 == '평생학교(중)-2년6학기','학교종류명'] = '중학교'
df.loc[df.학교종류명 == '평생학교(고)-2년6학기','학교종류명'] = '고등학교'
df.loc[df.학교종류명 == '방송통신고등학교','학교종류명'] = '고등학교'

In [ ]:
# 지오코딩 (주소를 통해 위도, 경도 좌표 얻기)

def geocoding(address):
    geo = geo_local.geocode(address)

    return geo.latitude, geo.longitude

In [ ]:
geocoding('돈화문로11가길')

(37.5759933, 126.9899017)

In [ ]:
lat, long = [], []
for i in tqdm(range(len(df))):
    try: 
        lat_i, long_i = geocoding(df.도로명주소[i])

    except AttributeError:
        lat_i, long_i = 0.0, 0.0
    lat.append(lat_i)
    long.append(long_i)

df['lat'] = lat
df['long'] = long

100%|██████████| 3852/3852 [32:04<00:00,  2.00it/s]


In [ ]:
df.loc[df.lat == 0.0 , :]

,학교종류명,도로명주소,lat,long
77,중학교,서울특별시 동작구 사당로2길 2-19,0.0,0.0
97,중학교,서울특별시 동작구 상도로13라길 22,0.0,0.0
143,초등학교,서울특별시 송파구 위례북로2길 12,0.0,0.0
144,중학교,서울특별시 강동구 아리수로93다길 1,0.0,0.0
145,초등학교,서울특별시 강동구 아리수로93다길 1,0.0,0.0
...,...,...,...,...
3673,고등학교,서울특별시 서초구 반포대로27길 29,0.0,0.0
3674,고등학교,서울특별시 서초구 반포대로27길 29,0.0,0.0
3675,고등학교,서울특별시 서초구 반포대로27길 29,0.0,0.0
3676,고등학교,서울특별시 서초구 반포대로27길 29,0.0,0.0


In [ ]:
df.to_csv('./data/seoul_schools_location.csv', index=False) # File Saving